In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from pathlib import Path
import os
import matplotlib.colors as mcolors
from matplotlib.ticker import MultipleLocator, FuncFormatter
from typing import Dict, List, Tuple

# 尝试加载自定义样式
try:
    from general_pic_setup import setup_mpl_single2
    setup_mpl_single2()
except ImportError:
    pass

# === 全局配置 ===
LEG_LEFT_X  = 0.681
LEG_RIGHT_X = 0.245
LEG_Y       = 0.979
NATURE_COLORS = [
    '#E64B35', "#6917C2", '#00A087', '#3C5488',
    '#F39B7F', '#8491B4', '#91D1C2', '#DC0000',
    '#7E6148', '#B09C85', '#E18727', '#20854E',
    '#0072B5', '#BC3C29', '#6F99AD',
]

mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'

def get_l2_to_l1_map(config_path: Path) -> Dict[str, str]:
    """构建 L2名称 到 L1分类 的严格映射表"""
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    return {
        item["L2"]: item["L1"] 
        for item in config.get("level_mapping", {}).values() 
        if item.get("L2") and item.get("L1")
    }

def lighten_color(color: str, amount: float = 0.7) -> tuple:
    """颜色变浅工具"""
    c = mcolors.to_rgb(color)
    return tuple([c[i] + (1 - c[i]) * amount for i in range(3)])

def custom_sort_key(col: str) -> Tuple[int, int, str]:
    """保留特定的列排序逻辑"""
    col_str = str(col)
    if 'Transport' in col_str: return (0, 0, col_str)
    if 'Electricity' in col_str: return (0, 1, col_str)
    if 'Industry' in col_str: return (0, 2, col_str)
    if 'Buildings' in col_str: return (0, 3, col_str)
    return (1, 0, col_str)

def plot_single_metric(input_file: Path, output_file: Path, metric_name: str, l2_to_l1_map: Dict[str, str]) -> None:
    """
    绘制指定指标的趋势图 (严格字典分类)。
    """
    # 1. 数据准备
    df = pd.read_parquet(input_file)
    df = df[(df['TIME_PERIOD'] >= 2005) & (df['TIME_PERIOD'] <= 2023)].sort_values('TIME_PERIOD')
    
    # 排除非数据列
    data_cols = [c for c in df.columns if c not in ('REF_AREA', 'TIME_PERIOD')]
    df_avg = df.groupby('TIME_PERIOD')[data_cols].mean()

    # 2. 严格分类
    cols_map = {'Incentive-based': [], 'Regulatory': [], 'Commitment-based': [], 'Research and Development (R&D)': []}
    
    for col in data_cols:
        l1_type = l2_to_l1_map.get(col)
        if l1_type in cols_map:
            cols_map[l1_type].append(col)
    
    # 3. 绘图参数
    total_l2 = sum(len(v) for v in cols_map.values())
    ave_y = 1 / total_l2 if (total_l2 > 0 and metric_name == 'Breadth') else 0.0
    y_max = max(0.2, df_avg.max().max() * 1.15) if not df_avg.empty else 0.2

    fig, axes = plt.subplots(3, 1, figsize=(16.15, 18.36), sharex=True, sharey=True)

    def _setup_ax(ax):
        ax.set_ylim(0, y_max)
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x)}'))
        ax.tick_params(axis='x', rotation=45)
        plt.setp(ax.xaxis.get_majorticklabels(), ha='center')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_ylabel('Proportion / Breadth' if metric_name == 'Breadth' else 'Intensity Score')

    def _draw_overlays(ax):
        if metric_name == 'Breadth':
            ax.axhline(ave_y, color="black", ls='--', lw=3, alpha=0.85, zorder=1, dashes=(3, 2))
            ax.annotate(f'Ave=1/{total_l2}', xy=(2023.9, ave_y), xytext=(8, 0), 
                        textcoords='offset points', ha='left', va='center', fontsize=15,
                        bbox=dict(boxstyle='round,pad=0.25', fc='white', ec='none', alpha=0.8))
        for y in [2009, 2015, 2020]:
            ax.axvline(y, color='#4A4A4A', ls='--', lw=2, alpha=0.6, zorder=0, dashes=(3, 2))

    def _plot_lines(ax, cols, title):
        handles, labels = [], []
        for i, col in enumerate(sorted(cols, key=custom_sort_key)):
            color = NATURE_COLORS[i % len(NATURE_COLORS)]
            line, = ax.plot(df_avg.index, df_avg[col], color=color, marker='o', lw=1.8, ms=15,
                            mfc=lighten_color(color), mec=color, mew=1.8, label=col, zorder=5)
            handles.append(line)
            labels.append(col)
        ax.set_title(title, pad=15)
        _draw_overlays(ax)
        _setup_ax(ax)
        return handles, labels

    def _add_legend(ax, handles, labels):
        # 修复 PRDD 换行问题
        fixed_labels = [l.replace("Public Research, Development and Demonstration", 
                                  "Public Research, Development\nand Demonstration") for l in labels]
        ax.legend(handles, fixed_labels, loc='upper left', bbox_to_anchor=(1.02, 1), 
                  frameon=False, fontsize=15, ncol=1, markerscale=0.73)

    # 4. 执行绘制
    h1, l1 = _plot_lines(axes[0], cols_map['Incentive-based'], "Incentive-based")
    _add_legend(axes[0], h1, l1)

    h2, l2 = _plot_lines(axes[1], cols_map['Regulatory'], "Regulatory")
    _add_legend(axes[1], h2, l2)

    # 混合图 (Commitment + R&D)
    ax3 = axes[2]
    hc, lc = _plot_lines(ax3, cols_map['Commitment-based'], "") # 先画背景
    
    # 手动叠加 R&D
    hr, lr = [], []
    offset = len(hc)
    for i, col in enumerate(sorted(cols_map['Research and Development (R&D)'], key=custom_sort_key)):
        color = NATURE_COLORS[(offset + i) % len(NATURE_COLORS)]
        line, = ax3.plot(df_avg.index, df_avg[col], color=color, marker='o', lw=1.8, ms=15,
                         mfc=lighten_color(color), mec=color, mew=1.8, label=col, zorder=5)
        hr.append(line)
        lr.append(col)
    
    ax3.set_title("Commitment-based & Research and Development (R&D)", pad=15)
    axes[2].set_xlabel('Year')
    _add_legend(ax3, hc + hr, lc + lr)

    plt.subplots_adjust(hspace=0.2, right=0.7)
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    print(f"✅ Generated: {output_file.name}")
    plt.close()

if __name__ == "__main__":
    base_dir = Path.cwd().parent
    data_dir = base_dir / "data"
    
    # 严格模式：必须加载 Config
    l2_to_l1 = get_l2_to_l1_map(data_dir / "config_mappings.json")
    
    plot_single_metric(data_dir/"2-1-country_breadth.parquet", 
                       data_dir/"2-2-global_trends_breadth.png", "Breadth", l2_to_l1)
                       
    plot_single_metric(data_dir/"2-1-country_intensity.parquet", 
                       data_dir/"2-2-global_trends_intensity.png", "Intensity", l2_to_l1)

✅ Generated: 2-2-global_trends_breadth.png
✅ Generated: 2-2-global_trends_intensity.png
